# Physics-Informed Neural Networks (PINN) 求解 Burgers 方程

## 1. 物理背景
我们要求解的是一维 Burgers 方程，它是流体力学中对流扩散方程的简化形式，常用于模拟激波（Shock Waves）的形成。

**方程形式：**
$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = \nu \frac{\partial^2 u}{\partial x^2}
$$

其中：
* $u(x, t)$：流体速度
* $\nu$：运动粘度系数（本例取 $\nu = 0.01/\pi$）
* 域：$x \in [-1, 1], \quad t \in [0, 1]$

**初始与边界条件：**
* **IC ($t=0$):** $u(x, 0) = -\sin(\pi x)$
* **BC ($x=\pm 1$):** $u(-1, t) = u(1, t) = 0$

## 2. 核心数学推导：自动微分与张量缩并

在 PINN 的代码实现中，最核心的一行代码是：
```python
grads = torch.autograd.grad(u, inputs, torch.ones_like(u), create_graph=True)[0]
```

为了理解这一步发生了什么，我们需要从张量分析的角度来拆解 **Vector-Jacobian Product (VJP)** 的过程。

### 2.1 定义变量与形状
假设我们有 $N$ 个采样点（例如 $N=2000$）：
* **输入 (Input)** $\mathbf{X}$：形状为 $[N, 2]$。第 $q$ 行第 $m$ 列元素 $X_{qm}$ 代表第 $q$ 个样本的第 $m$ 个特征（$m=1$为$x$, $m=2$为$t$）。
* **输出 (Output)** $\mathbf{u}$：形状为 $[N, 1]$。第 $p$ 行元素 $u_p$ 代表第 $p$ 个样本的预测速度。

### 2.2 隐形的三阶雅可比张量
理论上，输出 $\mathbf{u}$ 对输入 $\mathbf{X}$ 的导数是一个三阶张量 $\mathcal{J}$，形状为 $[N, N, 2]$：
$$
\mathcal{J}_{pqm} = \frac{\partial u_p}{\partial X_{qm}}
$$

利用神经网络的 **样本独立性 (Batch Independence)**：第 $p$ 个样本的输出只取决于第 $p$ 个样本的输入。当 $p \neq q$ 时，导数为 0。我们可以引入克罗内克符号 $\delta_{pq}$：

$$
\mathcal{J}_{pqm} = \frac{\partial u_p}{\partial X_{pm}} \cdot \delta_{pq}
$$

这是一个**稀疏张量**，只有对角线平面（$p=q$）上有值。

### 2.3 `ones_like(u)` 与张量“拍扁” (Flattening)
我们在 `grad` 函数中传入的 `grad_outputs` 是 $\mathbf{v} = \text{ones\_like}(u)$，这是一个形状为 $[N, 1]$ 的全 1 向量。
PyTorch 执行的是 **张量缩并 (Tensor Contraction)** 操作：

$$
R_{qm} = \sum_{p=1}^{N} v_p \cdot \mathcal{J}_{pqm}
$$

由于 $v_p = 1$，且 $\delta_{pq}$ 只有在 $p=q$ 时为 1，求和号 $\sum_p$ 会把所有 $p \neq q$ 的项（都是0）过滤掉，只保留 $p=q$ 的项。

**几何直观：**
这相当于用手掌（向量 $\mathbf{v}$）沿着输出样本维度（$p$轴）垂直向下压，把三阶张量 $\mathcal{J}$ **“拍扁”** 成了二维矩阵。

### 2.4 最终结果
$$
R_{qm} = \frac{\partial u_q}{\partial X_{qm}}
$$
结果 $\mathbf{R}$ 的形状是 $[N, 2]$，恰好对应了我们需要计算的 $u_x$ 和 $u_t$。这也是为什么我们需要后续进行切片操作 `[:, 0:1]` 和 `[:, 1:2]` 的原因。

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

# --- 配置 ---
# 使用 GPU 如果可用，否则使用 CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NU = 0.01 / torch.pi  # 粘度系数

In [ ]:
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        # 输入 2 维 (x, t)，输出 1 维 (u)
        self.net = nn.Sequential(
            nn.Linear(2, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)
        )
    
    def forward(self, x):
        return self.net(x)

# 初始化模型与优化器
model = PINN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def get_pde_data(n=2000):
    """生成物理方程所需的内部采样点 (Collocation Points)"""
    # 1. torch.rand 生成 [0, 1)
    # 2. * 2 - 1 将范围映射到 [-1, 1)
    x = (torch.rand(n, 1) * 2 - 1).to(device)
    t = (torch.rand(n, 1) * 1).to(device)
    
    # 【关键】拼接成 [N, 2]，并开启梯度追踪
    inputs = torch.cat([x, t], dim=1).requires_grad_(True)
    return inputs

In [ ]:
print("开始训练 Burgers' Equation...")

loss_history = []

for step in range(5001):
    
    # --- Part A: PDE Loss (物理方程残差) ---
    inputs = get_pde_data()
    u = model(inputs)
    
    # 第一次求导：得到 [u_x, u_t]
    # grad_outputs=torch.ones_like(u) 即上文提到的权重向量 v
    grads = torch.autograd.grad(u, inputs, torch.ones_like(u), create_graph=True)[0]
    
    # 切片分离导数
    u_x = grads[:, 0:1]
    u_t = grads[:, 1:2]
    
    # 第二次求导：得到 u_xx
    grads_2 = torch.autograd.grad(u_x, inputs, torch.ones_like(u_x), create_graph=True)[0]
    u_xx = grads_2[:, 0:1]
    
    # Burgers 方程残差: f = u_t + u*u_x - nu*u_xx
    f = u_t + u * u_x - NU * u_xx
    loss_pde = torch.mean(f**2)
    
    # --- Part B: Initial Condition Loss (IC) ---
    # t = 0, x ∈ [-1, 1], u = -sin(πx)
    x_ic = (torch.rand(500, 1) * 2 - 1).to(device)
    t_ic = torch.zeros(500, 1).to(device)
    inputs_ic = torch.cat([x_ic, t_ic], dim=1)
    
    u_ic_pred = model(inputs_ic)
    u_ic_true = -torch.sin(torch.pi * x_ic)
    loss_ic = torch.mean((u_ic_pred - u_ic_true)**2)
    
    # --- Part C: Boundary Condition Loss (BC) ---
    # x = ±1, t ∈ [0, 1], u = 0
    t_bc = torch.rand(500, 1).to(device)
    x_bc_left = -1 * torch.ones(500, 1).to(device)
    x_bc_right = 1 * torch.ones(500, 1).to(device)
    
    inputs_bc = torch.cat([
        torch.cat([x_bc_left, t_bc], dim=1),
        torch.cat([x_bc_right, t_bc], dim=1)
    ], dim=0)
    
    u_bc_pred = model(inputs_bc)
    loss_bc = torch.mean(u_bc_pred**2) # u should be 0
    
    # --- Part D: Backpropagation ---
    loss = loss_pde + loss_ic + loss_bc
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    loss_history.append(loss.item())
    
    if step % 500 == 0:
        print(f"Step {step}: Total Loss={loss.item():.6f} "
              f"(PDE={loss_pde.item():.5f}, IC={loss_ic.item():.5f}, BC={loss_bc.item():.5f})")

print("训练完成！")

In [ ]:
# --- 绘图准备 ---
x_np = np.linspace(-1, 1, 100)
t_np = np.linspace(0, 1, 100)
X, T = np.meshgrid(x_np, t_np) # 生成网格用于画图

# 展平并拼接，转为 Tensor
x_flat = X.flatten()[:, None]
t_flat = T.flatten()[:, None]
inputs_test = torch.from_numpy(np.hstack((x_flat, t_flat))).float().to(device)

# 预测
u_pred = model(inputs_test).cpu().detach().numpy()
U = u_pred.reshape(100, 100)

# --- 绘制热力图 ---
plt.figure(figsize=(10, 6))
plt.pcolormesh(T, X, U, cmap='jet', shading='auto')
cbar = plt.colorbar()
cbar.set_label('u(x, t)')
plt.xlabel('t (Time)')
plt.ylabel('x (Space)')
plt.title("Burgers' Equation Solution via PINN")

# 验证激波位置
plt.text(0.5, 0.5, 'Shock Formation', color='white', ha='center', fontweight='bold')
plt.show()

# --- 绘制 Loss 曲线 ---
plt.figure(figsize=(6, 4))
plt.plot(loss_history)
plt.yscale('log')
plt.title("Training Loss (Log Scale)")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.show()